In [1]:
from weather import *
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from os import mkdir
from os.path import isdir, isfile
from accidents_montreal import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import Row
import math
from io import StringIO

# New for testing weather

# old for development and experimentation

In [ ]:
t= test[0]
lat, long = t
year, month, day = (2006,5,2)
hour = 0
stations = get_stations(lat, long, year, month, day)
weighted_average_num = 0
weighted_average_denum = 0
weathers = list()
cols=list()
for station in stations:
    identity=station[0]
    cache_file_path = f'data/weather/s{identity}_{year}_{month}.h5'
    if isfile(cache_file_path):
        df = pd.read_hdf(cache_file_path, key='w')
    else:
        url = (f'http://climate.weather.gc.ca/climate_data/bulk_data_e.html?'
               f'format=csv&stationID={identity}&Year={year}&Month={month}&Day={day}&'
               f'timeframe=1&submit=Download+Data')
        print(url)
        csvfile = get(url).text
        with StringIO(csvfile) as csvfile:
            skip_header(csvfile)

            df = pd.read_csv(csvfile,
                             index_col='Date/Time', parse_dates=['Date/Time'])

        if not isdir('data/weather/'):
            mkdir('data/weather/')
        df.to_hdf(cache_file_path, key='w')
    
    s=df.loc[f'{year}-{month}-{day} {hour}:00'] 
    s.loc["station_denom"] = station[1]
    """s['Temp (°C)'] = s.at['Temp (°C)']/station[1]
    print(s['Temp (°C)'])"""
    if len(cols) == 0:
        cols = df.columns.values.tolist() + ["station_denom"]
    weathers.append(s)

weathers = pd.DataFrame(weathers, columns=cols)


In [ ]:

temperatures = weathers[['Temp (°C)', 'station_denom']] \
    .dropna() \
    .apply(lambda row: pd.Series([row[0]/row[1], 1/row[1]]), axis=1) 
temperatures[0].sum() / temperatures[1].sum()

In [ ]:
numeric_cols=[col for col in list(wheathers.dtypes.index) if wheathers.dtypes[col] in ['int64','float64']]
non_numeric_cols = [col for col in list(wheathers.dtypes.index) if col not in numeric_cols+['Weather']]
numeric_wheathers = wheathers.loc[:,numeric_cols]
means = numeric_wheathers.mean()
print(means)

In [ ]:
non_num_wheathers = wheathers.loc[:,non_numeric_cols] 
weather = wheathers.loc[:,'Weather'].dropna().drop_duplicates().values.tolist()
non_num_wheathers.iat[5,2]='A'

def get_majority_vote(col):
    vals = col.dropna().drop_duplicates().tolist()
    if len(vals) > 0:
        e= max(set(vals), key=vals.count)
        return [v for v in vals if vals.count(e)==vals.count(v)]
    else:
        return ''
    
non_num_wheathers = non_num_wheathers.apply(lambda col:small_func(col), axis=0)
a=non_num_wheathers.index.values.tolist()
b=non_num_wheathers.values.tolist()
c=means.index.values.tolist()
d=means.values.tolist()
indexes = a + c + ['Weather']
vals = b + d + [weather]

In [ ]:
Row(**dict(zip(indexes, vals)))